## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('/datasets/data.csv')
data.info()
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4  

### Вывод

Различается количество ненулевых данных в столбцах. Стаж в днях имеет тип вещественного числа, он отрицательный. Регистр в столбце об образовании тоже скачет. Данные о браке, цели с одним смыслом записаны по разному. Данные требуют обработки пропусков и изменения типа данных на соответствующий. 

### Шаг 2. Предобработка данных

### Обработка пропусков

Первый столбец с пропусками - это столбец со стажем. Прежде чем, к нему приходить, убедимся, что в столбце с количеством детей стоят адекватные значения.

In [2]:
print(data['children'].value_counts())


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


0, 1, 2 и 3 выглядят репрезентативно, но 20 и -1 явно выбиваются. Далее стоят 4 и 5, тоже выглядят репрезентативно. -1 могло взяться из-за неверного сбора данных из анкет с тире, где указано, например "Количество детей - 1". Значений 20 еще больше. Может быть стоит отнести их к значению 0. Возможная причина появления - ввод на островной клавиатуре, где для клавиши 0, 2  - единственная соседняя цифра.

In [3]:
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 0
print(data['children'].value_counts())




0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


Это больше похоже на реальность. Перейдем к стажу. Разберемся с отрицательным стажем.

In [4]:
print(data[data['days_employed'] <= 0]['days_employed'].count())

15906


15906 клиентов с отрицательным стажем. Пусть он станет положительным.

In [5]:
data.loc[data['days_employed'] < 0, 'days_employed'] = -data.loc[:, 'days_employed']
print(data[data['days_employed'] < 0]['days_employed'].count())

0


Теперь у нас 0 клиентов с отрицательным рабочим стажем, но мы помним, что стаж в днях у нас все еще вещественного типа.
Разберемся с пропусками в стаже и доходе. 

In [6]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


2174 клиента не имеют данных о стаже и доходе, вряд ли они безработные, если речь идет о кредитных клиентах. Это количественные переменные, которые точно ненулевые, поэтому я заменю их на характерные. 

In [7]:
print('Средний доход {:.2f}'.format(data['total_income'].mean()))
print('Медиана дохода {:.2f}'.format(data['total_income'].median()))
print('Средний стаж {:.2f}'.format(data['days_employed'].mean()))
print('Медиана стажа {:.2f}'.format(data['days_employed'].median()))

Средний доход 167422.30
Медиана дохода 145017.94
Средний стаж 66914.73
Медиана стажа 2194.22


Среднее дохода и стажа(!) выше медианы - распределение смещено вправо, логичнее доход будет заменить медианным значением. 

In [8]:
income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value = income_median)
days_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value = days_median)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Идем далее - возраст клиентов.

In [9]:
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


101 клиент с возрастом 0 - тоже можно заменить на средний. При этом оставить целочисленным

In [10]:
age_mean = data['dob_years'].mean()
data.loc[data['dob_years'] == 0 , 'dob_years'] = age_mean
print(data['dob_years'].value_counts())


35.00000    617
40.00000    609
41.00000    607
34.00000    603
38.00000    598
42.00000    597
33.00000    581
39.00000    573
31.00000    560
36.00000    555
44.00000    547
29.00000    545
30.00000    540
48.00000    538
37.00000    537
50.00000    514
43.00000    513
32.00000    510
49.00000    508
28.00000    503
45.00000    497
27.00000    493
56.00000    487
52.00000    484
47.00000    480
54.00000    479
46.00000    475
58.00000    461
57.00000    460
53.00000    459
51.00000    448
59.00000    444
55.00000    443
26.00000    408
60.00000    377
25.00000    357
61.00000    355
62.00000    352
63.00000    269
64.00000    265
24.00000    264
23.00000    254
65.00000    194
66.00000    183
22.00000    183
67.00000    167
21.00000    111
43.29338    101
68.00000     99
69.00000     85
70.00000     65
71.00000     58
20.00000     51
72.00000     33
19.00000     14
73.00000      8
74.00000      6
75.00000      1
Name: dob_years, dtype: int64


Тип очевидно стал float, заменим на int позже. Далее образование.

In [11]:
print(data['education'].value_counts())


среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Так, переведем все в нижний регистр.

In [12]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())
print(data['education_id'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


Отлично, тут все ок. Переходим к семейному положению.

In [13]:
print(data['family_status'].value_counts())
print(data['family_status_id'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


Все ок. Посмотрим на половую принадлежность.

In [14]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Одно значение точно стоит выбросить.

In [15]:
data = data[data['gender'] != 'XNA']
print(data['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


### Вывод

В процессе работы выявились некоторые недостатки в системе сбора данных, которые приводят к пропускам или нехарактерным значениям. Если сбор данных производится в электронном виде, то во многих столбцах стоило бы не предоставлять клиенту самому записывать данные в строку, а выбирать из заранее приготовленного списка. Это сделает данные единообразными.

### Замена типа данных

Заменим тип данных для дней стажа - они должны быть целочисленными. Заменим их методом astype, он позволит нам сделать их int'ми.

In [16]:

data['days_employed']=data['days_employed'].astype('int')
data['dob_years']=data['dob_years'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Вывод

Теперь типы данных соответствуют.

### Обработка дубликатов

Посмотрим есть ли дубликаты и сколько их

In [17]:
print(data.duplicated().sum())

71


71 дубликат, причем если проверить датасет на дубликаты сразу после открытия, то их будет 54. Появились они после работы с типами данных и регистром, возможно анкета или кредитная заявка отправлялась человеком несколько раз, и каждый раз он писал по-разному, так и занеслось в базу. Мы можем удалить полные дубликаты методом drop_duplicates.

In [18]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

0


### Вывод

Теперь полные дубликаты в начальной таблице удалены, но посмотрим, что будет если лемматезировать столбец с целью.

### Лемматизация

Напишем функцию с истользованием Pymystem, которая обнаруживает в лемматизированной строке нужное нам слово и оставляет только его. Жилье и недвижимость объединим в одну категорию. Применим эту функцию к столбцу 'purpose'. Потом проверим на дубликаты, так как тут от вариации формулировки они тоже могут появиться.

In [19]:
m=Mystem()
def lemma(goal):
    lemmas = m.lemmatize(goal)
    if "свадьба" in lemmas:
        return "свадьба"
    elif "образование" in lemmas:
        return "образование"
    elif "жилье" in lemmas or 'недвижимость' in lemmas:
        return "жилье"
    elif "автомобиль" in lemmas:
        return "автомобиль"
    return 'no'
data['purpose'] = data['purpose'].apply(lemma)

print(data['purpose'].value_counts())
print(data.duplicated().sum())

жилье          10810
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64
339


339 дубликатов появилось исключительно из-за разницы формулировки. Возможно, человек брал несколько кредитов под одну цель и каждый раз писал в анкете по-разному. Тем не менее мы их удалим.

In [20]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

0


### Вывод

Кредиты на жилье - самые популярные. Много дублиатов вскрывается при лемматизации.

### Категоризация данных

Нас спрашивают про уровень дохода в итоговых вопросах. Поэтому стоит категоризировать клиентов по доходам. У нас нет валюты или страны, где работает банк. Поэтому придется работать с категориями дохода типа "низкий", "средний","выше среднего", "высокий". Сделаем это по квантилям с шагом 0.25.

In [21]:

def set_income(income):
    if income < data['total_income'].quantile(q=0.25):
        return "низкий"
    elif  data['total_income'].quantile(q=0.25) < income < data['total_income'].quantile(q=0.5):
        return "средний"
    elif data['total_income'].quantile(q=0.5) < income < data['total_income'].quantile(q=0.75):
        return "выше среднего"
    return "высокий"
data['income_q'] = data['total_income'].apply(set_income)

### Вывод

Отлично, теперь мы сможем ответить на вопросы, касающиеся уровня дохода.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
Сгруппируем данные и узнаем процентное соотношение количество клиентов, имевших задолженности по разным группам.

In [22]:
print(data.groupby('children')['debt'].sum()/data.groupby('children')['debt'].count()*100)
print(data['children'].value_counts())

children
0     7.690647
1     9.276631
2     9.514468
3     8.206687
4    10.000000
5     0.000000
Name: debt, dtype: float64
0    13900
1     4797
2     2039
3      329
4       40
5        9
Name: children, dtype: int64


### Вывод

При болшем количестве детей выше доля должников, но для клиентов с 3, 4 и 5 детьми это не действует. Однако количество таких клиентов существенно меньше, что делает эти данные нерепрезентативными. Но эти клиенты должны приниматься во внимание, возможно другие данные показали бы бОльшу потенциальну опасность.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
print(data.groupby('family_status')['debt'].sum()/data.groupby('family_status')['debt'].count()*100)

family_status
Не женат / не замужем    9.841954
в разводе                7.124895
вдовец / вдова           6.687898
гражданский брак         9.412906
женат / замужем          7.694856
Name: debt, dtype: float64


### Вывод

Однозначно можно выделить высокий по сравнению с остальными уровень выплат в срок у людей, кто был в браке. Это группы "в разводе" и "вдовец / вдова". Можно предположить, что люди более сознательно подходят к кредитным обязательствам в случае потери партнера. Самый высокий уровень невыплат у не женатых и пар в гражданском браке. Для этих групп граница достаточно размытая. Такие клиенты отвечая на вопрос о семейном положении могут указать любую из этих двух категорий. Хотя высокий уровеень невыплат сохраняется в обоих группах, возможно из-за неготовности принять на себя ответственнойсть как в семейной жизни, так и в финансовой.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
print(data.groupby('income_q')['debt'].sum()/data.groupby('income_q')['debt'].count()*100)

income_q
высокий          7.722885
выше среднего    8.896473
низкий           7.880280
средний          8.826206
Name: debt, dtype: float64


### Вывод

На первый взгляд зависимости между уровнем дохода и возвратом кредита в срок нет.

- Как разные цели кредита влияют на его возврат в срок?

In [25]:
print(data.groupby('purpose')['debt'].sum()/data.groupby('purpose')['debt'].count()*100)

purpose
автомобиль     9.410112
жилье          7.387439
образование    9.334006
свадьба        8.065915
Name: debt, dtype: float64


### Вывод

Если кредит на жилье, то у него самая высокая вероятность выплаты. Это происходит из-за крайней необходимости продукта. Если человек его не выплатит - он рискует остаться без крыши над головой. Самые невыплачиваемые кредиты - на автомобиль и на образование. От этих продуктов легко отказаться и не выплатить кредит. Можно разбить автомобиль и решить, что и не за чем платить кредит. Если клиент не тянет образование или решает "забить" на образование, то тоже может решить не платить. Информация о размерах таких кредитов дала бы больше информации.

### Шаг 4. Общий вывод

Наиболее важными показателями для скоринговой модели станут цель кредита и семейное положение - они наиболее репрезентативно оценивают ответственность клиента. Интересно что уровень невыплат в независимости от категории клиентов не превышает 10%.